In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors

## Read html to csv

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df.to_csv()
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [3]:
cols=[x for i,x in enumerate(df.index) if df.iat[i,1]=='Not assigned']
df1= df.drop(cols,axis=0)
#df1=df[~df['Borough'].isin(['Not assigned'])] ~代表取反
#df1.head()
df1.reset_index(drop=True,inplace=True)

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
cols1=[ i for i,x in enumerate(df1.index) if df1.iat[i,2] == 'Not assigned']
for i in cols1:
    df1.replace(df1.iloc[i,2],df1.iloc[i,1],inplace=True)


## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
df1.shape

(103, 3)

## Build the dataframe with coordinate

In [6]:
df_cor = pd.read_csv('http://cocl.us/Geospatial_data')
df_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df2=pd.merge(df1,df_cor,how='inner',on=['Postal Code'])
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Create map of Toronto using latitude and longitude values

In [8]:
map_can = folium.Map(location=[43.7, -79.25], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)
    
map_can

## Cluster those location by K-means using the Latitude and Londtitude

In [9]:

X= df2.values[:,3:]


In [10]:

k_means = KMeans(init = "k-means++", n_clusters = 3, n_init = 10)
k_means.fit(X)
k_means_labels = k_means.labels_
k_means_labels


array([1, 1, 0, 2, 0, 2, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 1, 0, 2,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0,
       1, 0, 2, 0, 0, 2, 2, 1, 0, 2, 0, 0, 2, 2, 1, 0, 2, 0, 0, 2, 2, 1,
       0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 2, 2, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0,
       2, 2, 1, 0, 0, 2, 2, 1, 0, 0, 2, 0, 0, 2, 2])

In [11]:
# add clustering labels
df2.insert(0, 'Cluster Labels', k_means_labels)
df2


,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,0,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,0,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,2,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
# create map
map_clusters = folium.Map(location=[43.7, -79.25], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighbourhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters